In [2]:
import sys
!{sys.executable} -m pip install opencv-python

     -------------------------------------- 35.6/35.6 MB 317.8 kB/s eta 0:00:00


In [43]:
import cv2
import numpy as np

# Create the haar cascade
faceCascade = cv2.CascadeClassifier("./data/haarcascade_frontalface_default.xml")

# Read the image
image = cv2.imread("./data/well_lit.png")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
lab = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Detect faces in the image
faces = faceCascade.detectMultiScale(
    gray,
    scaleFactor=1.1,
    minNeighbors=5,
    minSize=(30, 30)
)


In [44]:
# Draw a rectangle around the face and get the corresponding area
for (x, y, w, h) in faces:
    face = image[y:y+h,x:x+w]
    face_gray= gray[y:y+h,x:x+w]
    face_rect_lab = lab[y:y+h,x:x+w]
    face_rect_hsv = hsv[y:y+h,x:x+w]

cv2.imshow("Face Found",image)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imshow("Face Mask LAB",face_rect_lab)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imshow("Face Mask HSV",face_rect_hsv)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [51]:
from scipy.signal import convolve2d

# Get skin mask
def elipse_mask(a,b):
    return ((a-143.0)/6.5)**2 + ((b-148.0)/12)**2

def hsv_mask(h,s):
    return (s>=0.25 and s<=0.75) and h<=0.095


def skin_detection():
    skin_mask = np.zeros((face.shape[0],face.shape[1]))
    avg_mask = np.ones((5,5))/25
    a_mean = convolve2d(face_rect_lab[:,:,1],avg_mask,mode='same',boundary='symm')
    b_mean = convolve2d(face_rect_lab[:,:,2],avg_mask,mode='same',boundary='symm')
    ellipse_test = np.vectorize(lambda a,b: 1 if elipse_mask(a,b) < 1 else 0)
    skin_mask = ellipse_test(a_mean,b_mean)
    print(avg_mask, a_mean,b_mean)
    hsv_test = np.vectorize(lambda h,s,m: 1 if m == 1 and hsv_mask(h,s) else 0)
    skin_mask = hsv_test(face_rect_hsv[:,:,0]/179.0,face_rect_hsv[:,:,0]/255.0,skin_mask)
    #large_ellipse_test = np.vectorize(lambda m,a,b: 1 if elipse_mask(a,b) < 1.25 and m == 1 else 0)
    #skin_mask = large_ellipse_test(skin_mask,a_mean,b_mean)
    return skin_mask.astype(np.uint8)
    
skin_mask = skin_detection()
print(skin_mask.shape)
print(face_gray.shape)
skin = cv2.bitwise_and(face, face, mask = skin_mask)
cv2.imshow("Skin Found",np.hstack([face,skin]))
cv2.waitKey(0)
cv2.destroyAllWindows()

[[0.04 0.04 0.04 0.04 0.04]
 [0.04 0.04 0.04 0.04 0.04]
 [0.04 0.04 0.04 0.04 0.04]
 [0.04 0.04 0.04 0.04 0.04]
 [0.04 0.04 0.04 0.04 0.04]] [[130.08 130.08 130.28 ... 130.32 130.68 130.84]
 [130.   130.   130.24 ... 130.64 130.92 131.  ]
 [130.   130.04 130.28 ... 131.2  131.4  131.36]
 ...
 [129.   129.   129.   ... 129.48 129.8  130.  ]
 [129.   129.   129.   ... 129.44 129.8  130.  ]
 [129.   129.   129.   ... 129.4  129.8  130.  ]] [[133.88 133.96 134.04 ... 135.8  135.76 135.68]
 [133.92 133.96 134.04 ... 135.6  135.64 135.72]
 [133.88 133.88 133.96 ... 135.76 135.84 136.08]
 ...
 [128.   128.   128.   ... 127.56 127.2  127.  ]
 [128.   128.   128.   ... 127.6  127.2  127.  ]
 [128.   128.   128.   ... 127.6  127.2  127.  ]]
(395, 395)
(395, 395)


In [ ]:
"""
Algorithm 1. CorrectFace(I,F,Mskin)
Require: I – input image (luminance channel).
Require: F – detected face rectangle.
Require: Mskin – skin mask.
Ensure: Iout – adjusted image (luminance channel).
1: // Perform edge-preserving base/detail decomposition
2: (Base,Detail) = W LSFilter(I)
3: Iout = Base
4: // Sidelight correction
5: S = F Mskin // skin pixels inside F
6: H = Smoothed histogram of intensities in Iout[S]
7: if H is bimodal (sidelit face) then
8: d = intensity of dark mode in H
9: b = intensity of bright mode in H
10: m = intensity at local minimum between d and b
11: f = b−d m−d
12: A = adjustment map scaling by f every pixel ∈ S
with intensity ≤ m
13: Apply edge-aware constraint propagation to A
14: Iout = Iout ·A // pixelwise multiplication
15: end if
16: // Exposure Correction
17: p = 75th percentile of face skin pixels S
18: if p < 120 (underexposed face) then
19: f = 120+p
2p ; ensure 1 ≤ f ≤ 2
20: A = adjustment map scaling by f every pixel ∈ S
21: Apply edge-aware constraint propagation to A
22: Iout = Iout ·A // pixelwise multiplication
23: end if
24: Iout+ = Detail
"""